
# Monasca Bootcamp
## Hands on Lab
---
### [Roland Hochmuth](https://www.linkedin.com/in/rolandhochmuth)

company: Hewlett Packard Enterprise

email: <roland.hochmuth@hpe.com>

### [Michael Hoppal](https://www.linkedin.com/in/hoppalmichael)

company: Hewlett Packard Enterprise

email: <michael.jam.hoppal@hpe.com>

# The rest of the Monasca team in Austin

* Deklan Dieterly (Hewlett Packard Enterprise)

* Brad Klein (TWC)

* Ryan Bak (TWC)

* Syd Logan (Broadcom)

* Witek Bedyk (FUJITSU Enabling Software Technology GmbH)

* Kojo Nakazono (Fujitsu)

* Shinya Kawabata (NEC)

* Fabio Giannetti (Cisco)

# Running the iPython notebook
---
This is an iPython/Jupyter notebook that you can run on your own. You will require the following installed on your system:

* iPython

    * See, https://ipython.org/install.html

* An OpenStack DevStack install with the Monasca DevStack plugin installed.

    * See, https://github.com/openstack/monasca-api/tree/master/devstack

* The following Python libraries:

    * pandas
    
    * numpy 

    * plotly
    
    * cufflinks

    * python-monascaclient
    
    * spur 

# Agenda
---
* What is Monasca?
* Architecture
* API, CLI (python-monascaclient) and client hands-on
* Agent hands-on
* Developing
* Current status
* What next?
* Horizon, Grafana 2 Demo

# What is Monasca?
---

* Monitoring-as-a-Service (at-scale) for OpenStack based on a first-class RESTful API
    * Highly performant, scalable and fault-tolerant
    * Authentication and multi-tenancy using Keystone
* Store and query metrics measurements and statistics
* Multiple ways to slice and dice the metrics and alarms based on filters and sorting options
* Create alarms and receive notifications based on state transitions
* Store and query alarm state history
* An extensible platform for monitoring components
* Used for operations and MaaS use cases.

# Architecture
---
![Monasca Architecture](metrics-architecture.png)

# Horizon
---
* A monitoring panel has been added to Horizon that supports various CRUD operations in the Monasca API.

![Horizon](horizon.png)

# Grafana 2.0
---
* TWC has added support for Monasca as a data source in Grafana 2.

* See the pull requests at:

    * https://github.com/grafana/grafana/pull/3963: Adds Keystone authentication. Note, this pull request will be merged in Grafana 3.0.
    
    * https://github.com/twc-openstack/grafana-plugins
    
![Grafana 2.0](grafana.png)

# Logging-as-a-Service
---
* Support for Logging-as-a-Service

* Repo: https://github.com/openstack/monasca-log-api

* Spec: https://github.com/openstack/monasca-log-api/blob/master/docs/monasca-log-api-spec.md

* Log Management with Monasca: At the Austin summit. See, https://www.openstack.org/summit/austin-2016/summit-schedule/events/7252

# Integration with other OpenStack projects
---
* Ceilometer
    * Collection and storage. Adds monitoring of OpenStack resources to Monasca and Monasca as a storage driver backen to Ceilometer.
    * See https://github.com/openstack/monasca-ceilometer
    * Ceilometer + Monasca = Ceilosca: https://www.openstack.org/summit/tokyo-2015/videos/presentation/ceilometer-monascaceilosca
* Heat Auto-scaling
    * https://www.openstack.org/summit/tokyo-2015/videos/presentation/auto-scaling-cloud-infrastructure-and-applications
* Broadview
    * Physical network switch monitoring
    * https://github.com/openstack?utf8=%E2%9C%93&query=broadview
* Congress
    * Policy management
* Vitrage: Starting to look at integration.
* Neutron: Starting to investigate how to do network monitoring in OpenStack.
* Rally

# Import libraries
---
Let's first import some libraries used in the rest of the notebook.

In [ ]:
import datetime
import time

# Import libraries use for visualization and analysis
import pandas as pd
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
import cufflinks as cf

# Import the Monasca and Keystone clients
from monascaclient import client
from monascaclient import ksclient

# Import library to execute remote commands for monasca-agent demo
import spur

# Initialize the Keystone and Monasca Client
---
We'll be connecting to the DevStack VM running on your local system and using the `mini-mon` project and username.

In [ ]:
KEYSTONE_URL = 'http://192.168.10.6:5000/v3'
PROJECT_NAME = 'mini-mon'
USERNAME = 'mini-mon'
PASSWORD = 'password'

In [ ]:
# Authenticate to Keystone
keystone_client = ksclient.KSClient(auth_url=KEYSTONE_URL, username=USERNAME, password=PASSWORD)

# Create the Monasca client
monasca_client = client.Client('2_0', keystone_client.monasca_url, token=keystone_client.token)

# Initialize environment variables to use the Monasca CLI
%env OS_PROJECT_NAME=$PROJECT_NAME
%env OS_PASSWORD=$PASSWORD
%env OS_AUTH_URL=$KEYSTONE_URL
%env OS_USERNAME=$USERNAME

# Initialize Plotly
---

We'll be using Plotly for displaying some graphs later on in this notebook for measurements and statistics returned by Monasca. Let's set Plotly to off-line mode.

In [ ]:
WIDTH = 800
HEIGHT = 600

cf.offline.go_offline()

# Initialize spur so we can run commands from Jupyter

In [ ]:
# Get Vagrant private key path by running "vagrant ssh-config" and copying IdentityFile here
vagrant_private_key_path = "/Users/rolandhochmuth/Documents/repos/openstack/monasca-api.java/devstack/.vagrant/machines/default/virtualbox/private_key"
shell = spur.SshShell(hostname="192.168.10.6",
                      username="vagrant",
                      missing_host_key=spur.ssh.MissingHostKey.accept,
                      private_key_file=vagrant_private_key_path)
monasca_agent_help = shell.run(["/opt/monasca-agent/bin/monasca-setup", "-h"])

# Sync up the clock. Note the VM clock might be out of sync after wake from suspend
shell.run(['sudo', 'ntpdate', 'pool.ntp.org'])

# Using the API
---

The [Monasca API](https://github.com/openstack/monasca-api/blob/master/docs/monasca-api-spec.md) has the following resources:

* Versions

* Metrics

* Metrics Measurements

* Metrics Statistics

* Metrics Names

* Notification Methods

* Alarm Definitions

* Alarms

* Alarms Count

* Alarms State History

# Common concepts
---
Let's review some common concepts in the API before we get into the various resources and how to use them.

# Roles
---

Roles are used to control access to the API.

Currently, there are three roles in Monasca:

1. user: Allows the client access to all CRUD operations on the API.

2. agent: Can only POST metrics to the API.

3. delegate: Allows the client to POST or query metrics to/from other tenants/projects.

    * Used by the Monasca Agent to POST metrics to the owner tenant/project for an OpenStack resource.
    
Roles are created in Keystone and assigned to the `roles` in the monasca-api config file when deployed.

# Pagination
---

* The Monasca API implements a paging mechanism to allow users to `page` through result sets returned by the API.

* The paging mechanism is accomplished by specifying the query parameters in the URL:

    * offset: The ID of the last element in the previous result set.

    * limit: The number of elements to return.
    
        * Note, the max `limit` of the Monasca API is 10,000 elements.
    
        * If the number of elements returned in the result set equals the max limit as a query parameter or of the API then there are additional elements to page in.
    
* The response object also includes a `links` field which will have a `next` field if paging. 

# Pagination example

First, let's just list some measurements using a limit of 8

In [77]:
!monasca measurement-list --limit 8 cpu.user_perc 2016-04-23T15:52:26.000Z

+---------------+---------------------+--------------------------+--------------+------------+
| name          | dimensions          | timestamp                | value        | value_meta |
+---------------+---------------------+--------------------------+--------------+------------+
| cpu.user_perc | hostname: devstack  | 2016-04-23T15:52:26.000Z |       18.500 |            |
|               | service: monitoring | 2016-04-23T15:52:40.000Z |        4.600 |            |
|               |                     | 2016-04-23T15:52:55.000Z |        1.900 |            |
|               |                     | 2016-04-23T15:53:10.000Z |        1.900 |            |
|               |                     | 2016-04-23T15:53:25.000Z |        1.600 |            |
|               |                     | 2016-04-23T15:53:40.000Z |        2.100 |            |
|               |                     | 2016-04-23T15:53:55.000Z |        1.700 |            |
|               |                     | 2016-04-23

Now, let's list the measurements from the start of the list (no offset) using a limit of 2

In [78]:
!monasca measurement-list cpu.user_perc --limit 2 2016-04-23T15:52:26.000Z

+---------------+---------------------+--------------------------+--------------+------------+
| name          | dimensions          | timestamp                | value        | value_meta |
+---------------+---------------------+--------------------------+--------------+------------+
| cpu.user_perc | hostname: devstack  | 2016-04-23T15:52:26.000Z |       18.500 |            |
|               | service: monitoring | 2016-04-23T15:52:40.000Z |        4.600 |            |
+---------------+---------------------+--------------------------+--------------+------------+


Now, let's page in the next 2 measurements starting from the offset of the last element in the previous list

In [79]:
!monasca measurement-list cpu.user_perc --offset 2016-04-23T15:52:40.000Z --limit 2 2016-04-23T15:52:26.000Z

+---------------+---------------------+--------------------------+--------------+------------+
| name          | dimensions          | timestamp                | value        | value_meta |
+---------------+---------------------+--------------------------+--------------+------------+
| cpu.user_perc | hostname: devstack  | 2016-04-23T15:52:55.000Z |        1.900 |            |
|               | service: monitoring | 2016-04-23T15:53:10.000Z |        1.900 |            |
+---------------+---------------------+--------------------------+--------------+------------+


# Metrics
---

* name (string(255), required) - The name of the metric. Naming conventions for metric names:

    * lowercase
    
    * `.` to delimit groups
    
    * `_` (snake case) to delimit words, with a uit of measurement as the suffix.

* dimensions ({string(255): string(255)}, optional) - A dictionary consisting of (key, value) pairs used to uniquely identify a metric and slice and dice on.

    * Examples of dimension keys are the following: hostname, region, zone, service, component, process, ...

* timestamp (string, required) - The timestamp in milliseconds from the Epoch.

* value (float, required) - Value of the metric.

* value_meta ({string(255): string(2048)}, optional) - A dictionary consisting of (key, value) pairs used to describe the metric.

    * Examples: status_code, msg

* tenant_id: Tenant ID to create metrics on behalf of.

    * This parameter can be used to submit metrics from one tenant, to another.
    * Requires the delegate role.

# Dimensions
---

* A dictionary of (key, value) pairs that are used to uniquely identify a metric.

* Used to slice and dice metrics when querying.

* Examples: hostname, service, component, region zone, resource_id, ...

* Dimensions can be anything you want, but naming conventions should be adopted for consistency.

# Example metrics request body
---
In this example, we are reporting the status of a HTTP check as a binary value, 0 or 1, for a specific host. The API is down, and the status code and msg returned are returned as meta data.
```
{
	name: http_status,
	dimensions: {
		url: http://service.domain.com:80,
		region: uswest,
		zone: 1,
		service: compute
	}
	timestamp: 1461600900000, /* milliseconds */
	value: 1.0,
	value_meta: {
		status_code: 500,
		msg: Internal server error
	}
}
```

# Create metrics help
---

In [80]:
!monasca help metric-create

usage: monasca metric-create [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                             [--value-meta <KEY1=VALUE1,KEY2=VALUE2...>]
                             [--time <UNIX_TIMESTAMP>]
                             [--project-id <CROSS_PROJECT_ID>]
                             <METRIC_NAME> <METRIC_VALUE>

Create metric.

Positional arguments:
  <METRIC_NAME>         Name of the metric to create.
  <METRIC_VALUE>        Metric value.

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to create a metric dimension. This
                        can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --value-meta <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair for extra information about a value.
       

# Create a metric

In [81]:
!monasca metric-create --dimensions hostname=foo test_metric 1.0

Successfully created metric


# List metrics
* Lists all the unique metrics in the system

* A unique metric is identitifed by it's name and dimensions.

* Starttime, endtime, offset and limit paramaters are available.

* Note, if you are monitoring OpenStack resources, such as VMs, based on the amount of churn (VMs being created/destroyed) in the system and your retention policy, it is recommended to specify the starttime, to limit the amount of metrics returned, such that queries return in a reasonable amount of time.

# List Metrics Help
---


In [82]:
!monasca help metric-list

usage: monasca metric-list [--name <METRIC_NAME>]
                           [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                           [--starttime <UTC_START_TIME>]
                           [--endtime <UTC_END_TIME>]
                           [--offset <OFFSET LOCATION>]
                           [--limit <RETURN LIMIT>]

List metrics for this tenant.

Optional arguments:
  --name <METRIC_NAME>  Name of the metric to list.
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --starttime <UTC_START_TIME>
                        measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR Format: -120 (pre

In [83]:
!monasca metric-list --limit 10

+--------------------------------------+---------------------------------+
| name                                 | dimensions                      |
+--------------------------------------+---------------------------------+
| monasca.thread_count                 | hostname: devstack              |
|                                      | component: monasca-agent        |
|                                      | service: monitoring             |
| test_metric                          |                                 |
| io.read_req_sec                      | device: sda1                    |
|                                      | hostname: devstack              |
|                                      | mount_point: /                  |
|                                      | service: monitoring             |
| net.out_bytes_sec                    | device: eth0                    |
|                                      | hostname: devstack              |
|                        

# List metrics and filter on name, dimensions and starttime
---

In [84]:
!monasca metric-list --name cpu.user_perc --dimensions hostname=devstack,service=monitoring --starttime 100 --limit 10

+---------------+---------------------+
| name          | dimensions          |
+---------------+---------------------+
| cpu.user_perc | hostname: devstack  |
|               | service: monitoring |
+---------------+---------------------+


# Create a function to get metrics using the Monasca client
---

In [85]:
def get_metrics(names = [None], dimensions = None, limit=10):
    metrics = []
    for name in names:
        kwargs = {}
        if name is not None:
            kwargs['name'] = name
        if dimensions is not None:
            kwargs['dimensions'] = dimensions
        kwargs['limit'] = limit
        
        # Invoke the Monasca client
        metrics = metrics + monasca_client.metrics.list(**kwargs)
    return metrics

# Metrics Measurements
---
* Operations for returning measurements.

* Name and dimensions supplied as query parameters

* Startime, endtime, offset and limit supplied.

* Note:

    * Currently, only one set of measurements can be returned.
    
    * If multiple metrics are requested, the `merge_metrics` flag must be specified.
    
    * There are a few reviews in progress to support multiple metrics in a single request.

# Measurement list help
---

In [86]:
!monasca help measurement-list

usage: monasca measurement-list [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                [--endtime <UTC_END_TIME>]
                                [--offset <OFFSET LOCATION>]
                                [--limit <RETURN LIMIT>] [--merge_metrics]
                                <METRIC_NAME> <UTC_START_TIME>

List measurements for the specified metric.

Positional arguments:
  <METRIC_NAME>         Name of the metric to list measurements.
  <UTC_START_TIME>      measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR Format: -120 (previous 120
                        minutes).

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
             

# Measurement list
---

In [87]:
!monasca measurement-list --dimensions hostname=devstack --limit 5 cpu.user_perc -120

+---------------+---------------------+--------------------------+--------------+------------+
| name          | dimensions          | timestamp                | value        | value_meta |
+---------------+---------------------+--------------------------+--------------+------------+
| cpu.user_perc | hostname: devstack  | 2016-04-23T20:41:08.000Z |        1.500 |            |
|               | service: monitoring | 2016-04-23T20:41:23.000Z |        1.900 |            |
|               |                     | 2016-04-23T20:41:38.000Z |        1.700 |            |
|               |                     | 2016-04-23T20:41:53.000Z |        2.300 |            |
|               |                     | 2016-04-23T20:42:08.000Z |        1.900 |            |
+---------------+---------------------+--------------------------+--------------+------------+


# Create a function to get measurements using the Monasca Client
---

In [88]:
def get_measurements(metrics, start_time = None, end_time = None, limit=None):
    measurements = []
    
    if start_time == None:
        start_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=3600)
        start_time = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    if end_time == None:
        end_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=0)
        end_time = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
        
    for metric in metrics:
        kwargs = {}
        kwargs['name'] = metric['name']
        kwargs['dimensions'] = metric['dimensions']
        kwargs['start_time'] = start_time
        kwargs['end_time'] = end_time
        
        # Invoke the Monasca client
        measurements.append(monasca_client.metrics.list_measurements(**kwargs))
        
    return measurements

# Create a function to translate measurements to a Pandas DataFrame
---

In [89]:
def df_from_measurements(measurements):
    '''Returns a DataFrame given measurements'''
    measurement = measurements[0][0]
    m = np.array(measurement['measurements'])  
    timestamps = m[:, measurement['columns'].index('timestamp')]
    df = pd.DataFrame(index = timestamps)  
    
    for measurement in measurements:
        measure = measurement[0]
        m = np.array(measure['measurements'])
        name = measure['name']
        df[name] = m[:, measure['columns'].index('value')]     
    return df

# Query measurements using the Monasca client
---


In [92]:
metrics = get_metrics(names=['cpu.user_perc', 'cpu.system_perc'])
measurements = get_measurements(metrics)

# Display measurements using Plotly
---

In [93]:
df = df_from_measurements(measurements)
df.iplot(fill=True, shared_xaxes=True, xTitle='Date', yTitle='Value', title='Statistics')

# Metrics Statistics
---
Operations for returning statistics.

# Get statistics help
---

In [94]:
!monasca help metric-statistics

usage: monasca metric-statistics [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                 [--endtime <UTC_END_TIME>]
                                 [--period <PERIOD>]
                                 [--offset <OFFSET LOCATION>]
                                 [--limit <RETURN LIMIT>] [--merge_metrics]
                                 <METRIC_NAME> <STATISTICS> <UTC_START_TIME>

List measurement statistics for the specified metric.

Positional arguments:
  <METRIC_NAME>         Name of the metric to report measurement statistics.
  <STATISTICS>          Statistics is one or more (separated by commas) of
                        [AVG, MIN, MAX, COUNT, SUM].
  <UTC_START_TIME>      measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR Format: -120 (previous 120
                        minutes).

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
    

# Create a function to get statistics using the Monasca client
---

In [95]:
def get_statistics(metrics, statistics=['avg'], interval = 3600, start_time = None, end_time = None):
    statistics_list = []
    
    period = interval / 512
    
    if period < 60:
        period = 60
        
    period = period - period%60
    
    if start_time == None:
        start_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=interval)
        start_time = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    if end_time == None:
        end_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=0)
        end_time = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
        
    for metric in metrics:
        kwargs = {}
        kwargs['statistics'] = statistics
        kwargs['period'] = period
        kwargs['name'] = metric['name']
        kwargs['dimensions'] = metric['dimensions']
        kwargs['start_time'] = start_time
        kwargs['end_time'] = end_time
        
        # Query using the Monasca client
        response = monasca_client.metrics.list_statistics(**kwargs)
        
        statistics_list.append(response)
        
    return statistics_list

# Create a function to translate statistics to a Pandas DataFrame
---

In [96]:
def df_from_statistics(statistics, fn = 'avg', group_by=[]):
    '''Returns a DataFrame given statistics'''
    stat = statistics[0][0]
    m = np.array(stat['statistics'])  
    timestamps = m[:, stat['columns'].index('timestamp')]
    df = pd.DataFrame(index = timestamps)
    
    for statistic in statistics:
        stat = statistic[0]
        m = np.array(stat['statistics'])
        name = stat['name']
        dimensions = stat['dimensions']
        
        for group in group_by:
            if group in dimensions:
                name += '.' + dimensions[group]
            
        df[name] = m[:, stat['columns'].index(fn)]
    return df

# Query statistics using the Monasca client
---

In [97]:
metrics = get_metrics(names=['cpu.user_perc', 'cpu.idle_perc', 'cpu.stolen_perc', 'cpu.system_perc', 'cpu.wait_perc'])
statistics = get_statistics(metrics, ['avg'], 3600)


# Display statistics using Plotly
---

In [98]:
import plotly.graph_objs as go
df = df_from_statistics(statistics)
df.iplot(kind='line',
         fill=True,
         xTitle='Date',
         yTitle='Value',
         title='Statistics',
         layout=go.Layout(xaxis=dict(ticktext="%s", tickmode="array"))
)

In [99]:
#shell.run(["stress-ng", "-c 2 -i 1 -m 1 --vm-bytes 128M -t 60s"])

# Display boxplot of cpu utilization using Plotly

In [100]:
df.iplot(kind='box')

# Notification Methods
---
* Specify the name, type and address to send a notification to.

* Notification methods are associated with actions in alarms and are invoked when staate transitions occur.

* Supported notification methods are:

  * Email
  
  * PagerDuty
  
  * Webhooks

# Delete all existing notifications
Let's first delete all existing notifications to prepare the service for creating new ones.

In [101]:
notifications = monasca_client.notifications.list()

for notification in notifications:
    kwargs = {}
    kwargs['notification_id'] = notification['id']
    monasca_client.notifications.delete(**kwargs)    

# Create Notification Method Help
---

In [102]:
!monasca help notification-create

usage: monasca notification-create <NOTIFICATION_NAME> <TYPE> <ADDRESS>

Create notification.

Positional arguments:
  <NOTIFICATION_NAME>  Name of the notification to create.
  <TYPE>               The notification type. Type must be EMAIL, WEBHOOK, or
                       PAGERDUTY.
  <ADDRESS>            A valid EMAIL Address, URL, or SERVICE KEY.


# Create an email notification method

In [103]:
!monasca notification-create "Email Notification" EMAIL john.doe@domain.com

{
  "address": "john.doe@domain.com", 
  "type": "EMAIL", 
  "id": "f51e78ef-891a-4434-b8d6-4afe3f266580", 
  "links": [
    {
      "href": "http://192.168.10.6:8070/v2.0/notification-methods/f51e78ef-891a-4434-b8d6-4afe3f266580", 
      "rel": "self"
    }
  ], 
  "name": "Email Notification"
}


# Create a PagerDuty notification method
Copy the `Integration Key` from PagerDuty and use it as the address when creating the PagerDuty notification method.

![PagerDuty](pagerduty.png)

# Create a PagerDuty notification method

In [104]:
!monasca notification-create "Monasca Bootcamp PagerDuty Notification" PAGERDUTY b9b0db2b6d114e6aa4ddb6fc933f43be

{
  "address": "b9b0db2b6d114e6aa4ddb6fc933f43be", 
  "type": "PAGERDUTY", 
  "id": "36f81a8a-4687-45f8-8e21-b0832b74f33f", 
  "links": [
    {
      "href": "http://192.168.10.6:8070/v2.0/notification-methods/36f81a8a-4687-45f8-8e21-b0832b74f33f", 
      "rel": "self"
    }
  ], 
  "name": "Monasca Bootcamp PagerDuty Notification"
}


# List Notification Methods Help
---

In [105]:
!monasca help notification-list

usage: monasca notification-list [--sort-by <SORT BY FIELDS>]
                                 [--offset <OFFSET LOCATION>]
                                 [--limit <RETURN LIMIT>]

List notifications for this tenant.

Optional arguments:
  --sort-by <SORT BY FIELDS>
                        Fields to sort by as a comma separated list. Valid
                        values are id, name, type, address, created_at,
                        updated_at. Fields may be followed by "asc" or "desc",
                        ex "address desc", to set the direction of sorting.
  --offset <OFFSET LOCATION>
                        The offset used to paginate the return data.
  --limit <RETURN LIMIT>
                        The amount of data to be returned up to the API
                        maximum limit.


# List Notification Methods

In [106]:
!monasca notification-list

+-----------------------------------------+--------------------------------------+-----------+----------------------------------+
| name                                    | id                                   | type      | address                          |
+-----------------------------------------+--------------------------------------+-----------+----------------------------------+
| Monasca Bootcamp PagerDuty Notification | 36f81a8a-4687-45f8-8e21-b0832b74f33f | PAGERDUTY | b9b0db2b6d114e6aa4ddb6fc933f43be |
| Email Notification                      | f51e78ef-891a-4434-b8d6-4afe3f266580 | EMAIL     | john.doe@domain.com              |
+-----------------------------------------+--------------------------------------+-----------+----------------------------------+


# Alarm Definitions
---
* Operations for creating, reading updating and deleting alarm definitions.

    * GET, POST /v2.0/alarm-definitions

    * GET, PUT, PATCH, DELETE /v2.0/alarm-definitions{alarm-definition-id}

* Alarm definitions are templates that are used to automatically create alarms based on matching metric names and dimensions

    * The `match-by` paramater is used to match/group metrics together by dimension
    
    * e.g. `--match-by hostname` will create an alarm per unique hostname.

* One alarm definition can result in many alarms

* Simple grammar for creating compound alarm expressions:

   avg(cpu.user_perc{}) > 85 or avg(disk.read_ops{device=vda}, 120) > 1000


* Alarm state (ALARM, OK and UNDETERMINED)

* Actions (notification methods) associated with alarms for state transitions

* User assigned severity (LOW, MEDIUM, HIGH, CRITICAL)



# Delete existing alarm definitions
Let's first delete all existing alarm definitions to prepare the service for creating new ones.

In [107]:
alarm_definitions = monasca_client.alarm_definitions.list()

for definition in alarm_definitions:
    kwargs = {}
    kwargs['alarm_id'] = definition['id']
    monasca_client.alarm_definitions.delete(**kwargs)    

# Create Alarm defintion help

In [108]:
!monasca help alarm-definition-create

usage: monasca alarm-definition-create [--description <DESCRIPTION>]
                                       [--severity <SEVERITY>]
                                       [--match-by <DIMENSION_KEY1,DIMENSION_KEY2,...>]
                                       [--alarm-actions <NOTIFICATION-ID>]
                                       [--ok-actions <NOTIFICATION-ID>]
                                       [--undetermined-actions <NOTIFICATION-ID>]
                                       <ALARM_DEFINITION_NAME> <EXPRESSION>

Create an alarm definition.

Positional arguments:
  <ALARM_DEFINITION_NAME>
                        Name of the alarm definition to create.
  <EXPRESSION>          The alarm expression to evaluate. Quoted.

Optional arguments:
  --description <DESCRIPTION>
                        Description of the alarm.
  --severity <SEVERITY>
                        Severity is one of [LOW, MEDIUM, HIGH, CRITICAL].
  --match-by <DIMENSION_KEY1,DIMENSION_KEY2,...>
                   

# Create alarm definition
Create an alarm definition that triggers if the avg of CPU user percent is greater than 80% over 3 60 periods.

In [109]:
!monasca alarm-definition-create simple-alarm "max(cpu.user_perc{}, 60) > 80 times 3"

{
  "alarm_actions": null, 
  "ok_actions": [], 
  "description": "", 
  "links": [
    {
      "href": "http://192.168.10.6:8070/v2.0/alarm-definitions/f20290da-76e5-4cfa-94e0-a23e6d8ca2d7", 
      "rel": "self"
    }
  ], 
  "match_by": [], 
  "severity": "LOW", 
  "actions_enabled": true, 
  "undetermined_actions": [], 
  "expression": "max(cpu.user_perc{}, 60) > 80 times 3", 
  "id": "f20290da-76e5-4cfa-94e0-a23e6d8ca2d7", 
  "name": "simple-alarm"
}


# Create a compound alarm definition
An alarm definition that triggers if either the CPU user or system percent is greater than 80% over 3 60 second periods.

In [110]:
!monasca alarm-definition-create compound-alarm "avg(cpu.user_perc{}, 60) > 80 times 3 or avg(cpu.system_perc{}, 60) > 80 times 3"

{
  "alarm_actions": null, 
  "ok_actions": [], 
  "description": "", 
  "links": [
    {
      "href": "http://192.168.10.6:8070/v2.0/alarm-definitions/a3b97d3a-5dab-45c8-886a-1fb511b309e5", 
      "rel": "self"
    }
  ], 
  "match_by": [], 
  "severity": "LOW", 
  "actions_enabled": true, 
  "undetermined_actions": [], 
  "expression": "avg(cpu.user_perc{}, 60) > 80 times 3 or avg(cpu.system_perc{}, 60) > 80 times 3", 
  "id": "a3b97d3a-5dab-45c8-886a-1fb511b309e5", 
  "name": "compound-alarm"
}


# Alarm definition match_by

* As metrics are consumed by the Threshold Engine they are filtered and checked if they match the definitions that have been specified, based on the metric name and dimensions.

* If a match is found and it is the first one, a new alarm is created.

* If the match_by parameter is specified, metrics are grouped into alarms that match the specified dimensions.

# Let's create two alarm definitions
1. One with no match_by parameter
2. One with match_by set on hostname

In [111]:
!monasca alarm-definition-create "match name only" "max(test-metric{}) > 80"
!monasca alarm-definition-create --match-by hostname "match by hostname" "max(test-metric{}) > 80"

{
  "alarm_actions": null, 
  "ok_actions": [], 
  "description": "", 
  "links": [
    {
      "href": "http://192.168.10.6:8070/v2.0/alarm-definitions/9ff3db98-9bb8-4eff-953a-3a52f40262dd", 
      "rel": "self"
    }
  ], 
  "match_by": [], 
  "severity": "LOW", 
  "actions_enabled": true, 
  "undetermined_actions": [], 
  "expression": "max(test-metric{}) > 80", 
  "id": "9ff3db98-9bb8-4eff-953a-3a52f40262dd", 
  "name": "match name only"
}
{
  "alarm_actions": null, 
  "ok_actions": [], 
  "description": "", 
  "links": [
    {
      "href": "http://192.168.10.6:8070/v2.0/alarm-definitions/3923c12d-5882-4c0c-924b-d045e9c3b08b", 
      "rel": "self"
    }
  ], 
  "match_by": [
    "hostname"
  ], 
  "severity": "LOW", 
  "actions_enabled": true, 
  "undetermined_actions": [], 
  "expression": "max(test-metric{}) > 80", 
  "id": "3923c12d-5882-4c0c-924b-d045e9c3b08b", 
  "name": "match by hostname"
}


# Let's now send two metrics
1. One with a name of test-metric and a hostname set to foo
2. One with a name of test-metric and a hostname set to bar

In [112]:
!monasca metric-create --dimensions hostname=foo test-metric 0.0
!monasca metric-create --dimensions hostname=bar test-metric 0.0

Successfully created metric
Successfully created metric


# How many alarms are created?

In [113]:
!monasca alarm-list --metric-name "test-metric"

+--------------------------------------+--------------------------------------+-----------------------+-------------+-------------------+----------+--------------+-----------------+------+--------------------------+--------------------------+--------------------------+
| id                                   | alarm_definition_id                  | alarm_definition_name | metric_name | metric_dimensions | severity | state        | lifecycle_state | link | state_updated_timestamp  | updated_timestamp        | created_timestamp        |
+--------------------------------------+--------------------------------------+-----------------------+-------------+-------------------+----------+--------------+-----------------+------+--------------------------+--------------------------+--------------------------+
| 1103533c-e493-4b80-b18a-6b610757759d | 3923c12d-5882-4c0c-924b-d045e9c3b08b | match by hostname     | test-metric | hostname: foo     | LOW      | UNDETERMINED | None            | None | 2

Three alarms were created:

1. There is one alarm with name "match name only". It has two metrics that are associate with it.

2. There are two alarms with name "match by hostname". Each alarm has one metric that is associated with it, based on the hostnames of `foo` and `bar`.

# Alarms
---

* Alarms are created when incoming metrics match alarm definitions

* Requests

    * GET /v2.0/alarms

    * GET, PUT, PATH, DELETE /v2.0/alarms/{alarm-id}

* Query Parameters

    * alarm_definition_id (string, optional) - Alarm definition ID to filter by.

    * metric_name (string(255), optional) - Name of metric to filter by.

    * metric_dimensions ({string(255): string(255)}, optional) - Dimensions of metrics to filter by specified as a comma separated array of (key, value) pairs as `key1:value1,key1:value1, ...`

    * state (string, optional) - State of alarm to filter by, either `OK`, `ALARM` or `UNDETERMINED`.

    * state_updated_start_time (string, optional) - The start time in ISO 8601 combined date and time format in UTC.
    
    * sort-by: Fields to sort by
    
    * offset, limit


# Alarms list help

In [114]:
!monasca help alarm-list

usage: monasca alarm-list [--alarm-definition-id <ALARM_DEFINITION_ID>]
                          [--metric-name <METRIC_NAME>]
                          [--metric-dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                          [--state <ALARM_STATE>] [--severity <SEVERITY>]
                          [--state-updated-start-time <UTC_STATE_UPDATED_START>]
                          [--lifecycle-state <LIFECYCLE_STATE>]
                          [--link <LINK>] [--sort-by <SORT BY FIELDS>]
                          [--offset <OFFSET LOCATION>]
                          [--limit <RETURN LIMIT>]

List alarms for this tenant.

Optional arguments:
  --alarm-definition-id <ALARM_DEFINITION_ID>
                        The ID of the alarm definition.
  --metric-name <METRIC_NAME>
                        Name of the metric.
  --metric-dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified 

# Alarm list

In [115]:
!monasca alarm-list

+--------------------------------------+--------------------------------------+-----------------------+-------------+-------------------+----------+--------------+-----------------+------+--------------------------+--------------------------+--------------------------+
| id                                   | alarm_definition_id                  | alarm_definition_name | metric_name | metric_dimensions | severity | state        | lifecycle_state | link | state_updated_timestamp  | updated_timestamp        | created_timestamp        |
+--------------------------------------+--------------------------------------+-----------------------+-------------+-------------------+----------+--------------+-----------------+------+--------------------------+--------------------------+--------------------------+
| 1103533c-e493-4b80-b18a-6b610757759d | 3923c12d-5882-4c0c-924b-d045e9c3b08b | match by hostname     | test-metric | hostname: foo     | LOW      | UNDETERMINED | None            | None | 2

# Alarm Counts
---
* Provides the ability to query counts of alarms using a number of filter and group-by query parameters.

* Primarily used in summary/overview dashboards to show the number of alarms in the OK, ALARM and UNDETERMINED state.

* Queries are processed in-database, not client-side.

# OpsConsole Summary Dashboard
Counts of alarms are queried using the alarms count resource.

![OpsConsole](ops-console-dashboard.png)

In [116]:
!monasca help alarm-count

usage: monasca alarm-count [--alarm-definition-id <ALARM_DEFINITION_ID>]
                           [--metric-name <METRIC_NAME>]
                           [--metric-dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                           [--state <ALARM_STATE>] [--severity <SEVERITY>]
                           [--lifecycle-state <LIFECYCLE_STATE>]
                           [--link <LINK>] [--group-by <GROUP_BY>]
                           [--offset <OFFSET LOCATION>]
                           [--limit <RETURN LIMIT>]

Count alarms.

Optional arguments:
  --alarm-definition-id <ALARM_DEFINITION_ID>
                        The ID of the alarm definition.
  --metric-name <METRIC_NAME>
                        Name of the metric.
  --metric-dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensi

In [117]:
!monasca alarm-count --group-by state,dimension_name

+-------+--------------+----------------+
| count | state        | dimension_name |
+-------+--------------+----------------+
| 5     | UNDETERMINED | hostname       |
| 2     | UNDETERMINED | service        |
+-------+--------------+----------------+


# Alarm History
---
* Provides the ability to query the history of alarms and all state transitions.

* Used to understand if alarms are flapping as well as do root cause analysis (RCA) on alarms.

In [118]:
!monasca help alarm-history

usage: monasca alarm-history [--offset <OFFSET LOCATION>]
                             [--limit <RETURN LIMIT>]
                             <ALARM_ID>

Alarm state transition history.

Positional arguments:
  <ALARM_ID>            The ID of the alarm.

Optional arguments:
  --offset <OFFSET LOCATION>
                        The offset used to paginate the return data.
  --limit <RETURN LIMIT>
                        The amount of data to be returned up to the API
                        maximum limit.


In [119]:
!monasca help alarm-history-list

usage: monasca alarm-history-list [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                  [--starttime <UTC_START_TIME>]
                                  [--endtime <UTC_END_TIME>]
                                  [--offset <OFFSET LOCATION>]
                                  [--limit <RETURN LIMIT>]

List alarms state history.

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --starttime <UTC_START_TIME>
                        measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR format: -120 (previous 120
                        minutes).
  --endtime <UTC_END_TIME>

# List alarm history help
---

In [120]:
!monasca help alarm-history-list

usage: monasca alarm-history-list [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                  [--starttime <UTC_START_TIME>]
                                  [--endtime <UTC_END_TIME>]
                                  [--offset <OFFSET LOCATION>]
                                  [--limit <RETURN LIMIT>]

List alarms state history.

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --starttime <UTC_START_TIME>
                        measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR format: -120 (previous 120
                        minutes).
  --endtime <UTC_END_TIME>

# List alarm history

In [121]:
!monasca alarm-history-list

# List alarm history using the Monasca client
---

In [122]:
monasca_client.alarms.history_list(**{})

[]

# Agent
---
* A Python monitoring agent
* Push model
* Agent is installed on the systems that we want to monitor
* Collects metrics by running a set of collection plugins every X amount of seconds 
* Collection plugins are enabled by detection plugins
* Detection plugins generate yaml config files that the collection plugins read from
* The agent has a monasca-setup command line tool that helps configure the agent and run detection plugins

# Agent

* System metrics (cpu, memory, network, filesystem, …)

* Service metrics

* RabbitMQ, MySQL, Kafka, and many others

* Application metrics

    * Built-in statsd daemon

    * Python monasca-statsd library: Adds support for dimensions

* VM system metrics

* Active checks

    * HTTP status checks and response times
    
    * System up/down checks (ping and ssh)
    
* Support for Nagios and checkmk

* Extensible/Pluggable: Additional services can be easily added


# Monasca-setup options

In [123]:
monasca_agent_help = shell.run(["/opt/monasca-agent/bin/monasca-setup", "-h"])
print monasca_agent_help.output

usage: monasca-setup [-h] [-u USERNAME] [-p PASSWORD]
                     [--user_domain_id USER_DOMAIN_ID]
                     [--user_domain_name USER_DOMAIN_NAME]
                     [--keystone_url KEYSTONE_URL]
                     [--project_name PROJECT_NAME]
                     [--project_domain_id PROJECT_DOMAIN_ID]
                     [--project_domain_name PROJECT_DOMAIN_NAME]
                     [--project_id PROJECT_ID] [--monasca_url MONASCA_URL]
                     [--system_only]
                     [-d [DETECTION_PLUGINS [DETECTION_PLUGINS ...]]]
                     [-a DETECTION_ARGS] [--check_frequency CHECK_FREQUENCY]
                     [--dimensions DIMENSIONS] [--ca_file CA_FILE]
                     [--insecure INSECURE] [--config_dir CONFIG_DIR]
                     [--log_dir LOG_DIR] [--log_level LOG_LEVEL]
                     [--template_dir TEMPLATE_DIR] [--overwrite] [-r]
                     [--skip_enable] [--install_plugins_only] [--user USER

# Monasca-setup configuring agent

In [124]:
monasca_agent_setup_command = shell.run(["sudo", "cat", "/usr/local/bin/monasca-reconfigure"])
print monasca_agent_setup_command.output

#!/bin/sh
'/opt/monasca-agent/bin/monasca-setup' \
    -u 'monasca-agent' \
    -p 'password' \
     -s 'monitoring'  \
    --keystone_url 'http://192.168.10.6:35357/v3' \
    --project_name 'mini-mon' \
     --monasca_url 'http://192.168.10.6:8070/v2.0'  \
     \
     --check_frequency '15'  \
     \
     --log_level 'WARN'  \
    --overwrite \
    --system_only



# Monasca agent configuration file

In [125]:
monasca_agent_setup_command = shell.run(["sudo", "cat", "/etc/monasca/agent/agent.yaml"])
print monasca_agent_setup_command.output

Api:
  amplifier: 0
  backlog_send_rate: 1000
  ca_file: null
  insecure: false
  keystone_url: http://192.168.10.6:35357/v3
  max_buffer_size: 1000
  password: password
  project_domain_id: null
  project_domain_name: null
  project_id: null
  project_name: mini-mon
  url: http://192.168.10.6:8070/v2.0
  user_domain_id: null
  user_domain_name: null
  username: monasca-agent
Logging:
  collector_log_file: /var/log/monasca/agent/collector.log
  forwarder_log_file: /var/log/monasca/agent/forwarder.log
  log_level: WARN
  statsd_log_file: /var/log/monasca/agent/statsd.log
Main:
  check_freq: 15
  collector_restart_interval: 24
  dimensions:
    service: monitoring
  hostname: localhost
  sub_collection_warn: 6
Statsd:
  monasca_statsd_port: 8125



# Agent Detection Plugins

* Run after initial configuration is run
* List of avaiable plugins 


# Run Kafka Detection Plugin

In [126]:
monasca_agent_mysql_detection_run = shell.run(["sudo", "/opt/monasca-agent/bin/monasca-setup", "-d", "kafka"])
print monasca_agent_mysql_detection_run.stderr_output

INFO: 	Watching the kafka process.
INFO: 	Kafka found listening on 192.168.10.6:9092
INFO: 	Installing kafka_consumer plugin.
INFO: Configuring Kafka
INFO: No changes found for plugins ['Kafka'], skipping restart of Monasca Agent



# Query the Kafka consumer lag metrics

In [127]:
!monasca metric-list --name kafka.consumer_lag

+--------------------+---------------------------------------------------------+
| name               | dimensions                                              |
+--------------------+---------------------------------------------------------+
| kafka.consumer_lag | topic: window-1                                         |
|                    | hostname: devstack                                      |
|                    | component: kafka                                        |
|                    | service: kafka                                          |
|                    | consumer_group: monasca-transform-aggregate-transformer |
| kafka.consumer_lag | topic: window-0                                         |
|                    | hostname: devstack                                      |
|                    | component: kafka                                        |
|                    | service: kafka                                          |
|                    | consu

# Query the Kafka consumer lag measurements

In [128]:
!monasca measurement-list kafka.consumer_lag --dimensions consumer_group=1_metrics -10 --limit 10

+--------------------+---------------------------+--------------------------+--------------+------------+
| name               | dimensions                | timestamp                | value        | value_meta |
+--------------------+---------------------------+--------------------------+--------------+------------+
| kafka.consumer_lag | topic: metrics            | 2016-04-23T22:32:51.000Z |       66.000 |            |
|                    | hostname: devstack        | 2016-04-23T22:33:06.000Z |       34.000 |            |
|                    | component: kafka          | 2016-04-23T22:33:21.000Z |       68.000 |            |
|                    | service: kafka            | 2016-04-23T22:33:36.000Z |       68.000 |            |
|                    | consumer_group: 1_metrics | 2016-04-23T22:33:51.000Z |       66.000 |            |
|                    |                           | 2016-04-23T22:34:06.000Z |       34.000 |            |
|                    |                        

# Display using Plotly

In [130]:
metrics = get_metrics(names=['kafka.consumer_lag'])
statistics = get_statistics(metrics, ['avg'], 3600)
df = df_from_statistics(statistics, group_by=['consumer_group'])
df.iplot(kind='line', fill=True, xTitle='Date', yTitle='Value', title='Statistics')

# Display a boxplot using Plotly

In [132]:
df.iplot(kind='box')

# Example detection yaml configuration

In [133]:
monasca_agent_detection_conf_example = shell.run(["sudo", "cat", "/etc/monasca/agent/conf.d/kafka_consumer.yaml"])
print monasca_agent_detection_conf_example.output

init_config: null
instances:
- built_by: Kafka
  consumer_groups:
    1_alarm-state-transitions:
      alarm-state-transitions: []
    1_metrics:
      metrics: []
    monasca-notification:
      alarm-state-transitions: []
      retry-notifications: []
    monasca-transform-aggregate-transformer:
      window-0: []
      window-1: []
      window-2: []
      window-3: []
    monasca-transform-identity-transformer:
      metrics: []
    thresh-event:
      events: []
    thresh-metric:
      metrics: []
  kafka_connect_str: 192.168.10.6:9092
  name: 192.168.10.6:9092
  per_partition: false



# Developing and Testing
---

* Python and Java codebase

* Monasca DevStack Plugin

* Unit Tests

* Monasca Tempest Tests

* Monasca and OpenStack CI

# Monasca Repos
---
* Monasca is a micro-services message bus based architecture.
* Several repos:
    * monasca-api: both Python and Java  
    * monasca-persister: both Python and Java
    * monasca-thresh: Java
    * monasca-notification: Python
    * monasca-common: both Python and Java
    * monasca-agent: Python
    * monasca-statsd: Python
    * monasca-ui: Python
    * python-monascaclient: Python
    * puppet-monasca: Puppet
    * monasca-log-api

# Monasca DevStack Plugin
---

* DevStack is the primary developmement environment for OpenStack.

    * See http://docs.openstack.org/developer/devstack/

* The Monasca DevStack plugin installs the Monasca Service, Agent, Horizon Panel, and Grafana

* README at, https://github.com/openstack/monasca-api/tree/master/devstack

* Best way to get started is to install Vagrant using the Vagrantfile at, https://github.com/openstack/monasca-api/blob/master/devstack/Vagrantfile.

# Monasca Tempest Tests
---

* [Tempest](http://docs.openstack.org/developer/tempest) is the integration test suite for OpenStack.

* Tempest has an external [Test Plugin Interface](http://docs.openstack.org/developer/tempest/plugin.html) interface that enables anyone to integrate an external test suite.

* There is a [Monasca Tempest Plugin](https://github.com/openstack/monasca-api/tree/master/monasca_tempest_tests)

* Currently, there are around 150 Tempest Tests written for Monasca that run in around 5 minutes.

# Monasca and OpenStack CI
---

* Monasca is fully integrated in the OpenStack CI system.

* Gated jobs are run with each commit on both the Python and Java components through the normal gates and all the Monasca Tempest tests using the Monasca DevStack Plugin.

* Currently, Java is non-voting

# Monasca Weekly Meetings
---
Weekly on Wednesday at 1500 UTC in #openstack-meeting-3 (IRC webclient)

# What's new?
---
* Many enhancements for filtering and sorting resources that return arrays.

* Multiple metrics (in progress): Allow multiple metrics to be returned in a single request.

* Sporadic metrics (in progress): Supports event based metrics. Metrics that occur infrequently or when events occur.

* Periodic notifications (in progress): Send notifications more than once. Better support for Heat auto-scaling.

# Next Steps
---

* monasca-transform: A transform and aggregation micro-service for Monasca.

    * blueprint: https://blueprints.launchpad.net/monasca/+spec/monasca-transform
    
    * repo: https://github.com/openstack/monasca-transform

* monasca-analytics: An experimental micro-service for Monasca that does anomaly detection and alarm clustering/correlation.

* monasca-events

# Acknowledgments
---

* The OpenStack Technical Committee: Doug Hellman, Flavio Percoco, Thierry Carrez

* Hewlett Packard Enterprise

* Time Warner Cable

* Fujitsu

* Cisco

* NEC

* Cray

* SAP

# Resources
---

## Cufflinks
* https://plot.ly/ipython-notebooks/cufflinks/
* http://web.quant-platform.com/trial/yves/Plotly_Cufflinks.html

## ipython Slideshow/RISE
* https://github.com/damianavila/RISE
* http://www.slideviper.oquanta.info/tutorial/slideshow_tutorial_slides.html#/3
* http://www.damian.oquanta.info/posts/make-your-slides-with-ipython.html
* https://github.com/damianavila/slideviper_test/blob/gh-pages/tutorial/slideshow_tutorial.ipynb
* http://lab.hakim.se/reveal-js/#/themes

# Thank you
![Thank you](austin-thank-you.png)